<a href="https://colab.research.google.com/github/Ranya-Alghamdi-1/T5/blob/main/YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [32]:
!pip install opencv-python-headless ultralytics

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [33]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import drive
from IPython.display import Video
import shutil

In [34]:
# Mount Google Drive
drive.mount('/content/drive')

# Load the YOLO model
model_path = '/content/drive/MyDrive/T5/Week 6/5- Weekly Project (Sunday)/5- Weekly Project (Sunday)/Dataset/YOLO_Model.pt'
model = YOLO(model_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [38]:
# Load the video
video_path = '/content/drive/MyDrive/T5/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)
# Get video properties
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
print("Video's height: ", h)
print("Video's width: ", w)
print("Video's FPS: ", fps)

# Initialize video writer
output_video_path = "/content/Yolo_Weekly_Project.mp4"
video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

Video's height:  1080
Video's width:  1920
Video's FPS:  29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [40]:
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        frame_copy = frame.copy()

        results = model.predict(frame_copy)
        bounding_boxes = results[0].boxes

        # Draw the exit lines and regions
        cv2.polylines(frame_copy, [upper_exit_line], isClosed=True, color=(0, 0, 255), thickness=4)
        cv2.polylines(frame_copy, [left_exit_path], isClosed=True, color=(0, 0, 255), thickness=4)
        cv2.polylines(frame_copy, [lower_exit_line], isClosed=True, color=(0, 0, 255), thickness=4)
        cv2.polylines(frame_copy, [right_exit_path], isClosed=True, color=(0, 0, 255), thickness=4)

        for box in bounding_boxes.xyxy:
            x1, y1, x2, y2 = map(int, box[:4])

            # Check if the box contains confidence and class ID
            if len(box) > 4:
                conf = round(float(box[4]), 2)
            else:
                conf = None

            if len(box) > 5:
                class_id = int(box[5])
            else:
                class_id = None

            # Calculate center of the bounding box
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2

            # Draw bounding box and center
            cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.circle(frame_copy, (center_x, center_y), 5, (0, 255, 0), -1)

            # Display class ID, confidence, and center
            if conf is not None and class_id is not None:
                cv2.putText(frame_copy, f'ID: {class_id} Conf: {conf}', (x1, y1 - 10), text_font, text_scale, text_color, 2)
            cv2.putText(frame_copy, f'Center: ({center_x},{center_y})', (x1, y2 + 20), text_font, text_scale, text_color, 2)

            # Count vehicles based on their position relative to the thresholds
            if (center_x < left_threshold_x) and (600 > center_y > 400):
                vehicle_counter["left"] += 1
            elif (center_x > right_threshold_x) and (700 > center_y > 550):
                vehicle_counter["right"] += 1
            elif (center_y < upper_threshold_y) and (1100 > center_x > 1000):
                vehicle_counter["upper"] += 1
            elif (center_y > lower_threshold_y) and (800 > center_x > 700):
                vehicle_counter["lower"] += 1

        # Display vehicle count on the video frame
        for path, position in display_positions.items():
            cv2.rectangle(frame_copy, (position[0] - 10, position[1] - 25),
                          (position[0] + 460, position[1] + 10), rect_color, -1)
            cv2.putText(frame_copy, f'Vehicles in {path.capitalize()} Path: {vehicle_counter[path]}',
                        position, text_font, text_scale, text_color, 2, cv2.LINE_AA)

        # Display name on the video frame
        cv2.putText(frame_copy, name, name_position, name_font, name_font_scale, name_font_color, 2, cv2.LINE_AA)

        # Write the processed frame to output video
        video_writer.write(frame_copy)
    else:
        break



0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 62.9ms
Speed: 3.1ms preprocess, 62.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 62.9ms
Speed: 3.0ms preprocess, 62.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 62.8ms
Speed: 2.2ms preprocess, 62.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 45.7ms
Speed: 3.3ms preprocess, 45.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 45.7ms
Speed: 3.1ms preprocess, 45.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 40.6ms
Speed: 4.2ms preprocess, 40.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 40.7ms
Speed: 3.9ms preprocess, 40.7ms inferenc

The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
**I searched in google and github**


## Save and Submit Your Work

In [41]:

cap.release()
video_writer.release()

# Move the output video to Google Drive
destination_path = '/content/drive/MyDrive/T5/Yolo_Weekly_Project.mp4'
shutil.copy2(output_video_path, destination_path)

# Display the video in the notebook
Video(destination_path)


## **Second Copy for this task to save a video in google drive to avoid the problem I have with video shows**

Submit both the notebook and the output video